# 通过callbacks保存checkpoint-1

## 1. 基础代码

包含：导入包，加载数据，创建模型

In [ ]:
import tensorflow as tf
from tensorflow import keras

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

# 定义模型构建函数
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784, )),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.sparse_categorical_crossentropy,
                  metrics=['accuracy'])

    return model

# 创建模型
model = create_model()

## 2. 配置callbacks保存checkpoint文件

In [ ]:
checkpoint_path = "../saved_models/training_1/cp.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)

model.fit(
    train_images,
    train_labels,
    epochs=10,
    validation_data=(test_images, test_labels),
    callbacks=[cp_callback]
)

## 3. 创建一个新模型

### 3.1 使用新模型，直接评估测试数据

In [ ]:
new_model = create_model()
loss, acc = new_model.evaluate(test_images, test_labels)
print("未训练模型, 评估精度: {:5.2f}%".format(acc * 100))

### 3.2 新模型读取参数后，评估测试数据

In [ ]:
new_model.load_weights(checkpoint_path)
loss,acc = new_model.evaluate(test_images, test_labels)
print("模型读取参数后, 评估精度:: {:5.2f}%".format(acc * 100))